## Capture the image from camera 
Save the captured image as `sample.png`

In [ ]:
import cv2

capture = cv2.VideoCapture(0)

while True:
    ret,img = capture.read()
    
    if not ret:
        break
    cv2.imshow("Camera",img)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    if key & 0xFF == ord('c'): # capture the image
        cv2.imwrite('sample.png',img)
capture.release()
cv2.destroyAllWindows()

## Find the Focal Length of Camera 

Focal length of camera: 
https://imaging.nikon.com/lineup/dslr/basics/19/01.htm

12 mm -> 200mm

12 mm: wide angle

200mm: close angle


In [ ]:
import numpy as np 
import cv2

# Width of piece of A4 paper ~ 21 cm: 8.26772 inch
# Known distance = 1 m ~ 39.3701 inch

KNOWN_WIDTH = 8.26772
KNOWN_DISTANCE = 39.3701
image = cv2.imread('sample.png')

def find_marker(image):
    # convet the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)

    # find the contours in the edged image and keep the largest one;
    # we'll assume that this is our piece of paper in the image
    (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    c = max(cnts, key = cv2.contourArea)
    
    # compute the bounding box of the paper region and return it
    return cv2.minAreaRect(c)

def find_focal_length_camera(P,D,W):
    """
        Calculate the focal length of camera
        f = (P x D)/ W
            f: Focal length of camera
            P: Width of object in camera (pixels)
            D: Known distance from camera to piece of paper
            W: Width of object (centimeters)
    """
    focal_length = (P*D)/W
    print ("[INFO] Focal length of camera: {}".format(focal_length))
    return focal_length


marker = find_marker(image)
# Width of object in camera (pixels)
P = marker[1][0]
# Calculate the focal length of camera
focal_length = find_focal_length_camera(P,KNOWN_DISTANCE,KNOWN_WIDTH)

## Backup Step: Can not find the width pixel of object in camera?

Manually measure the width of the piece in camera

P = 296 (pixels)

In [1]:
# calculate again focal_length of camera

KNOWN_WIDTH = 8.26772
KNOWN_DISTANCE = 39.3701
P = 160

def find_focal_length_camera(P,D,W):
    """
        Calculate the focal length of camera
        f = (P x D)/ W
            f: Focal length of camera
            P: Width of object in camera (pixels)
            D: Known distance from camera to piece of paper
            W: Width of object (centimeters)
    """
    focal_length = (P*D)/W
    print ("[INFO] Focal length of camera: {}".format(focal_length))
    return focal_length

focal_length = find_focal_length_camera(P,KNOWN_DISTANCE,KNOWN_WIDTH)

[INFO] Focal length of camera: 761.9048540589183


## Visualize the captured piece of paper after find Contours 

Sometimes, we cannot find the accurate bounding box (piece of paper) in the image

We need to go to backup step.

In [ ]:
import numpy as np 
import cv2

image = cv2.imread('sample.png')

def show_object(image):
    # convet the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)

    # find the contours in the edged image and keep the largest one;
    # we'll assume that this is our piece of paper in the image
    (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    c = max(cnts, key = cv2.contourArea)
    
    # compute the bounding box of the paper region and return it
    marker = cv2.minAreaRect(c)
    
    # draw a bounding box around the image and display it
    box = np.int0(cv2.boxPoints(marker))
    
    cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
    cv2.imwrite('result.png',image)
    cv2.imshow("image", image)
    key = cv2.waitKey(0)
    if key & 0xFF == ord('q'):
        return
    cv2.destroyAllWindows()

# run showing object
show_object(image)

## Real Testing with Camera

Find the distance from face to camera

In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [8]:
import cv2
from maskDect.maskDetector import MaskDetector

# Init Face Detector
detector = MaskDetector('/mnt/49418012-cfa6-4af1-86d8-c0fb55ae6501/Gaze_Estimation/Gitlab_Code/src/graphs/face_mask_detection.params')

# 13-14 cm
WIDTH_FACE = 14 
FOCAL_LENGTH = 761

capture = cv2.VideoCapture(0)

def distance_to_camera(knownWidth, focalLength, perWidth):
    # compute and return the distance from the maker to the camera
    return (knownWidth * focalLength) / perWidth

while True:
    ret,img = capture.read()
    
    if not ret:
        break
    
    # face prediction
    bboxes,confs, _ = detector.detect_face(img,target_shape=(260,260))
    
    # Visualize
    if bboxes:
        for index,box in enumerate(bboxes):
            PIXEL_WIDTH_FACE = box[3] - box[1]
            distance = distance_to_camera(WIDTH_FACE,FOCAL_LENGTH,PIXEL_WIDTH_FACE)
            cv2.rectangle(img, (box[1], box[0]), (box[3], box[2]), (0,255,0), 2)
            cv2.putText(img, "%.2f cm" % (distance),(int(box[1]),int(box[0]-10)), cv2.FONT_HERSHEY_SIMPLEX,
        1.0, (0, 255, 0), 2)
            
    cv2.imshow("Camera",img)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    if key & 0xFF == ord('s'):
        cv2.waitKey(0)
        
capture.release()
cv2.destroyAllWindows()

## Sample program

In [ ]:
# import the necessary packages
import numpy as np
import cv2


def find_marker(image):
    # convet the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)

    # find the contours in the edged image and keep the largest one;
    # we'll assume that this is our piece of paper in the image
    (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    c = max(cnts, key = cv2.contourArea)
    
    # compute the bounding box of the paper region and return it
    return cv2.minAreaRect(c)

def distance_to_camera(knownWidth, focalLength, perWidth):
    # compute and return the distance from the maker to the camera
    return (knownWidth * focalLength) / perWidth


# initialize the known distance from the camera to the object, which
# in this case is 24 inches
KNOWN_DISTANCE = 24.0

# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 11.0

# initialize the list of images that we'll be using
IMAGE_PATHS = ["images/1.png", "images/1.png", "images/3.png"]

# loop over the images
for imagePath in IMAGE_PATHS:
    # load the image, find the marker in the image, then compute the
    # distance to the marker from the camera
    image = cv2.imread(imagePath)
    marker = find_marker(image)
    inches = distance_to_camera(KNOWN_WIDTH, focalLength, marker[1][0])

    # draw a bounding box around the image and display it
    box = np.int0(cv2.cv.BoxPoints(marker))
    cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
    cv2.putText(image, "%.2fft" % (inches / 12),
        (image.shape[1] - 200, image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
        2.0, (0, 255, 0), 3)
    cv2.imshow("image", image)
    cv2.waitKey(0)